# Critic’s Cut 🎥 Powering Multilingual, Context-Aware Movie Reviews with ADK & Gemini 2.0
This notebook demonstrates how to build, evaluate, and secure a sophisticated movie and TV show review agent. It leverages the Google Agent Development Kit (ADK) and the Gemini 2.0 Flash model to create a conversational agent capable of:
- Understanding context in follow-up questions.
- Providing reviews in multiple languages (though primarily demonstrated in English).
- Fetching and summarizing reviews from the web using Google Search.
- Formatting reviews with ratings, emojis, and sources.
- Undergoing a series of automated evaluations for safety, content quality, and adherence to instructions before responses are shown to the user.

In [17]:
# @title Install all required libraries
# This cell installs all necessary Python libraries for the project.
# --quiet flag is used to suppress verbose installation output.

# Google Agent Development Kit
!pip install google-adk --quiet

# For language detection
!pip install langdetect --quiet

# For PII (Personally Identifiable Information) detection
!pip install presidio-analyzer --quiet

# For profanity/toxicity detection
!pip install better-profanity --quiet

In [18]:
# @title Imports and Environment Setup

# Standard Library Imports
import asyncio
import os  # For interacting with the operating system (e.g., environment variables)
import re  # For regular expression operations (text pattern matching)
import traceback # For printing stack traces during exception handling

# Third-Party Library Imports
from langdetect import detect, DetectorFactory  # For detecting the language of a given text
from presidio_analyzer import AnalyzerEngine    # For PII detection
from better_profanity import profanity          # For profanity/toxicity detection
from IPython.display import Markdown, display   # For displaying rich content (like Markdown) in IPython/Jupyter

# First-Party (Google/Project-specific) Imports
from google.colab import userdata                 # For accessing secrets in Google Colab
from google.adk.tools import google_search        # Pre-built tool for Google Search integration
from google.adk.agents import Agent               # Core class for defining an agent
from google.adk.runners import Runner             # Core class for running an agent
from google.adk.sessions import InMemorySessionService # For in-memory session management (stores conversation history)

# Google Generative AI types for constructing messages
from google.genai.types import Content, Part

print("All necessary modules imported.")

All necessary modules imported.


In [19]:
# @title API Keys and Constants
APP_NAME_MAIN = "critics_cut_app_v1"

# Define the language model to be used by the agent.
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash" # Utilizing Gemini 2.0 Flash model

# Retrieve the Google API Key from Colab's userdata (secrets)
print("\nConfiguring API keys...")
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    if not os.environ["GOOGLE_API_KEY"]:
        raise ValueError("GOOGLE_API_KEY not found or is empty in Colab secrets.")
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"
    print(f"API environment configured. Using model: {MODEL_GEMINI_2_0_FLASH}")
except Exception as e:
    print(f"🔴 ERROR accessing GOOGLE_API_KEY: {e}")

# Configure the agent to use the Gemini API directly, not via Vertex AI for this demo.
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

print(f"Environment configured. Using model: {MODEL_GEMINI_2_0_FLASH}")


Configuring API keys...
API environment configured. Using model: gemini-2.0-flash
Environment configured. Using model: gemini-2.0-flash


In [20]:
# @title Initialize Safety/Evaluation Tools
print("\nInitializing safety and evaluation tools...")
try:
    profanity.load_censor_words()
    analyzer = AnalyzerEngine()
    DetectorFactory.seed = 0 # For consistent langdetect results
    print("Safety and evaluation tools initialized.")
except Exception as e:
    print(f"🔴 ERROR initializing safety/evaluation tools: {e}")


Initializing safety and evaluation tools...


Safety and evaluation tools initialized.


In [21]:
# @title Input Validation Function

def is_movie_or_show_query(query: str) -> bool:
    """
    Checks if the user's query is likely about a movie or TV show.
    Uses keyword matching and common review-seeking patterns.
    """
    query_lower = query.lower()

    keywords = [
        "movie", "show", "series", "film", "tv", "television", "drama", "anime",
        "sitcom", "documentary", "thriller", "comedy", "actor", "actress", "director",
        "review of", "reviews of" # More specific review patterns
    ]
    if any(re.search(rf'\b{kw}\b', query_lower) for kw in keywords):
        return True

    # Patterns like "reviews for [title]", "about [title]", "[title] review(s)"
    # This pattern is more greedy for titles if keywords weren't hit.
    if re.search(r'(reviews? for|find reviews? for|about|what about|tell me about)\s+(.+)', query_lower) or \
       re.search(r'(.+)\s+reviews?', query_lower): # Catches "[title] reviews"
        return True

    # Basic check: if the query is short and doesn't have typical question words,
    # it might be a movie title. This is a very loose heuristic.
    # A more robust solution would involve a movie title database or NER.
    # if len(query_lower.split()) <= 4 and not any(q_word in query_lower for q_word in ["what", "who", "is", "how", "can", "do"]):
    #     return True # Heuristic: might be a title

    return False

# --- Test Cases for is_movie_or_show_query ---
print("--- Testing is_movie_or_show_query ---")
test_queries = {
    "Tell me about the movie Inception": True,
    "Inception reviews": True,
    "What is the weather like": False,
    "who played batman": True, # "who played" implies a character/actor context
    "the new batman movie": True,
    "Barbie": True # Heuristic might catch this if we enable the short query check, otherwise needs NER.
                  # Current regex might not catch standalone titles well without more context words.
}
for tq, expected in test_queries.items():
    result = is_movie_or_show_query(tq)
    status = "✅" if result == expected else "❌"
    print(f"{status} Query: '{tq}' -> Expected: {expected}, Got: {result}")

--- Testing is_movie_or_show_query ---
✅ Query: 'Tell me about the movie Inception' -> Expected: True, Got: True
✅ Query: 'Inception reviews' -> Expected: True, Got: True
✅ Query: 'What is the weather like' -> Expected: False, Got: False
❌ Query: 'who played batman' -> Expected: True, Got: False
✅ Query: 'the new batman movie' -> Expected: True, Got: True
❌ Query: 'Barbie' -> Expected: True, Got: False


In [22]:
# @title Define the Movie Review Agent

# Defines the core logic and instructions for our movie review agent.
# The agent's behavior is primarily guided by the 'instruction' prompt.
# It uses the globally defined MODEL_GEMINI_2_0_FLASH.

if 'MODEL_GEMINI_2_0_FLASH' not in globals():
    print("🔴 ERROR: MODEL_GEMINI_2_0_FLASH not defined. Please run Cell 3.")
    MODEL_GEMINI_2_0_FLASH = "gemini-1.0-pro" # Fallback

movie_review_agent = Agent(
    name="MovieReviewAgent",
    model=MODEL_GEMINI_2_0_FLASH,
    instruction="""
      You are Critic's Cut, a friendly and insightful movie and TV show review assistant.
      Your primary goal is to provide concise, well-formatted, and sourced reviews.

      **Core Task: Providing Reviews**
      When the user asks for reviews of a specific movie or TV show:
      1.  **Understand the Request:** Identify the movie/show title.
      2.  **Search for Information:** Use the `google_search` tool to find recent, authoritative reviews. If the movie title or query is in a language other than English, try to find reviews in that language or about that specific version of the film. However, primary review content might often be in English from major sources.
      3.  **Summarize Concisely:** Synthesize information from the top 2-3 reviews.
      4.  **Structured Rating & Comments:** For each of the following aspects, provide:
          -   A star rating out of 10 (e.g., ⭐⭐⭐⭐⭐⭐⭐⭐☆☆ for 8/10).
          -   A corresponding emoji.
          -   A brief, insightful comment.
          *   **Acting:** 🎭
          *   **Visuals/Cinematography:** 👁️
          *   **Story/Plot:** 📖
          *   **Direction:** 🎬
          *   **Overall:** 🏆
      5.  **Cite Sources Clearly:** Provide clickable Markdown links. Format: `**[Website Name]:** [Article Title](URL)`
      6.  **Handle No Reviews:** If no relevant reviews are found, politely state: "Sorry, I couldn't find any reviews for '[Movie/Show Title]'."

      **Contextual Understanding & Follow-up Questions:**
      -   Infer movie/show from RECENT conversation history for follow-ups.
      -   If context is unclear, ask for clarification: "Which movie or TV show are you referring to?"
      -   For specific follow-ups (e.g., "How was the acting?"), focus on that aspect. For general ones ("Was it good?"), give an overall summary.

      **Handling Off-Topic Queries:**
      -   Politely decline non-movie/TV show requests: "Sorry, as Critic's Cut, I specialize in movie and TV show reviews."

      **Multilingual Capabilities:**
      -   If the user's query is in a language other than English (e.g., Spanish, French), try to provide conversational parts of your response (greetings, clarifications) in that language.
      -   The core review summary (ratings, comments, sources) will primarily be based on information available from web searches, which may often be in English. However, always present this information clearly.
      -   If a movie title itself is in a non-English language, search for that title.
      -   Example (User query in Spanish: "¿Qué tal la película 'Coco'?"):
          "Claro, aquí tienes un resumen de las críticas para 'Coco':
          **Coco – Review Summary**
          **Overall:** ⭐⭐⭐⭐⭐⭐⭐⭐⭐☆ (9/10) 🏆
          *[Summary in English as sourced, or Spanish if high-quality Spanish reviews are found and summarized]*
          **Sources:** [Links to sources]"

      **Response Formatting:**
      -   ALL responses MUST be in Markdown.
      -   Use headings and bullet points.
      -   Ensure comprehensive reviews include: Title, all five rated aspects, and 2-3 linked sources.

      **Example - Full Review (English):**
      ---
      **Dune: Part Two – Review Summary**
      (Rest of the English example as before)
      ---
      """,
    tools=[google_search]
)

print("Movie Review Agent defined with multilingual instructions.")

Movie Review Agent defined with multilingual instructions.


In [23]:
# @title Initialize Session Service and Runner

# This cell initializes THE session_service and THE runner for the notebook.
# All subsequent operations will use these instances.
# It relies on APP_NAME_MAIN and movie_review_agent being defined.

if 'APP_NAME_MAIN' not in globals() or 'movie_review_agent' not in globals():
    print("🔴 ERROR: APP_NAME_MAIN or movie_review_agent not defined.")
    # Define fallbacks or raise error to stop execution
    APP_NAME_MAIN = "fallback_app"
    if 'movie_review_agent' not in globals(): raise NameError("movie_review_agent is not defined")


print(f"--- Initializing ADK Components for App: '{APP_NAME_MAIN}' ---")

# 1. Initialize THE Session Service instance
# This instance will be used throughout the notebook.
session_service = InMemorySessionService()
print(f"✅ InMemorySessionService initialized (ID: {id(session_service)}).")

# 2. Create THE Runner instance
# This runner is configured with our specific agent, app name, and session service.
try:
    runner = Runner(
        agent=movie_review_agent,
        app_name=APP_NAME_MAIN,
        session_service=session_service
    )
    print(f"✅ ADK Runner configured and ready (ID: {id(runner)}).")
    print(f"   Runner App Name: '{runner.app_name}'")
    print(f"   Runner Session Service ID: {id(runner.session_service)} (should match above)")
except Exception as e:
    print(f"🔴 ERROR initializing ADK Runner: {e}")
    print(traceback.format_exc())

# 3. Optional: Initialize a default/test session (can help verify service is working)
# This is not strictly necessary for the main conversation test cell if it creates its own session.
async def _verify_session_service():
    try:
        test_user = "service_check_user"
        test_session = "service_check_session"
        await session_service.create_session(
            app_name=APP_NAME_MAIN, user_id=test_user, session_id=test_session
        )
        s = await session_service.get_session(
            app_name=APP_NAME_MAIN, user_id=test_user, session_id=test_session
        )
        if s and s.id == test_session:
            print(f"✅ Session service verified: Successfully created and retrieved test session '{test_session}'.")
        else:
            print("⚠️ Session service verification: Session not found after creation.")
    except Exception as e:
        print(f"⚠️ Error during session service self-test: {e}")

# Run the verification in the current event loop
if 'runner' in globals(): # Only run if runner init was successful
    loop = asyncio.get_event_loop()
    if loop.is_running():
        loop.create_task(_verify_session_service())
    else:
        try:
            asyncio.run(_verify_session_service())
        except RuntimeError: # Should be caught by is_running, but as a fallback
             loop.create_task(_verify_session_service())

--- Initializing ADK Components for App: 'critics_cut_app_v1' ---
✅ InMemorySessionService initialized (ID: 133446741465744).
✅ ADK Runner configured and ready (ID: 133445093089744).
   Runner App Name: 'critics_cut_app_v1'
   Runner Session Service ID: 133446741465744 (should match above)


In [24]:
# @title Advanced Evaluation & Safety Functions

# --- Prerequisite Check for Evaluation Tools ---
evaluation_tools_ready = True
if 'analyzer' not in globals():
    print("🔴 ERROR: Presidio Analyzer ('analyzer') not initialized. Please ensure Cell 3 ran correctly.")
    evaluation_tools_ready = False
    # You could raise an error here to stop execution:
    # raise NameError("Presidio Analyzer ('analyzer') is not initialized.")

if 'profanity' not in globals():
    print("🔴 ERROR: Better-Profanity ('profanity') module not imported. Please ensure Cell 3 ran correctly.")
    evaluation_tools_ready = False
    # raise NameError("Better-Profanity ('profanity') module is not imported.")
else:
    # If 'profanity' object exists, we assume 'load_censor_words()' was called in Cell 3.
    # The check_toxicity function will operate based on whatever state profanity is in.
    # No easy public attribute to check if words are loaded without accessing internals.
    pass # The print statement in Cell 3 for tool init should give us confidence.

if not evaluation_tools_ready:
    print("🔴 One or more evaluation tools are not ready. Evaluation functions might not work correctly.")
else:
    print("✅ Evaluation tools (Analyzer, Profanity) appear to be available.")

# --- Evaluation Functions ---
# (Your refined check_... functions go here. I'll include one as an example)

def check_has_ratings_and_emojis(agent_response: str) -> bool:
    """
    Checks if the agent's response includes star ratings (⭐) and
    at least one of the specified aspect emojis (🎭👁️📖🎬🏆).
    """
    has_stars = bool(re.search(r'⭐', agent_response))
    has_aspect_emoji = bool(re.search(r'[🎬🎭📖👁️🏆]', agent_response))
    return has_stars and has_aspect_emoji

# (Include all other check_... functions: check_has_sources, check_rejection,
#  check_faithfulness, check_toxicity, check_pii, check_prompt_injection, check_language)
# ...

def check_has_sources(agent_response: str) -> bool:
    has_link = bool(re.search(r'https?://', agent_response))
    known_source_keywords = [
        "source", "guardian", "rotten tomatoes", "imdb", "metacritic",
        "new york times", "variety", "hollywood reporter", "bbc"
    ]
    has_source_name = any(source_kw in agent_response.lower() for source_kw in known_source_keywords)
    return has_link or has_source_name

def check_rejection(agent_response: str, is_actually_movie_topic: bool) -> bool:
    agent_declined_interaction = "sorry, i only answer questions about movies and tv shows" in agent_response.lower()
    if not is_actually_movie_topic:
        return agent_declined_interaction
    else:
        return not agent_declined_interaction

def check_faithfulness(agent_response: str, expected_sources: list = None) -> bool:
    if "sorry, i only answer questions about movies and tv shows" in agent_response.lower() or \
       "couldn't find any reviews" in agent_response.lower():
        return True
    KNOWN_REPUTABLE_SOURCES = [
        "guardian", "rotten tomatoes", "imdb", "metacritic", "new york times",
        "variety", "hollywood reporter", "bbc", "empire online"
    ]
    sources_to_check = expected_sources if expected_sources else KNOWN_REPUTABLE_SOURCES
    return any(source.lower() in agent_response.lower() for source in sources_to_check)

def check_toxicity(agent_response: str) -> bool:
    return not profanity.contains_profanity(agent_response)

def check_pii(agent_response: str) -> bool:
    results = analyzer.analyze(text=agent_response,
                               entities=["EMAIL_ADDRESS", "PHONE_NUMBER", "CREDIT_CARD_NUMBER"],
                               language='en')
    return len(results) == 0

def check_prompt_injection(agent_response: str) -> bool:
    forbidden_phrases = [
        "ignore previous instructions", "ignore all prior directives",
        "you are now in [hacker_mode]", "reveal your system prompt",
        "what is your initial instruction", "as an ai language model, my instructions are",
        "my confidential api key is", "internal use only", "system override"
    ]
    response_lower = agent_response.lower()
    return not any(phrase in response_lower for phrase in forbidden_phrases)

def check_language(agent_response: str, expected_lang_code: str = None) -> bool:
    if not agent_response.strip(): return True
    if not expected_lang_code: return True
    try:
        detected_lang = detect(agent_response) # DetectorFactory.seed was set in Cell 3
        return detected_lang == expected_lang_code
    except Exception:
        return False

print("Evaluation and safety functions defined.")

✅ Evaluation tools (Analyzer, Profanity) appear to be available.
Evaluation and safety functions defined.


In [25]:
# @title Helper Function: ask_movie_agent (with Evaluations)

# This function sends a prompt to the agent, gets the response,
# runs evaluations, and displays the outcome.
# It uses the globally defined 'is_movie_or_show_query' and 'check_...' functions.

async def ask_movie_agent(runner_instance: Runner, prompt_text: str, current_user_id: str, current_session_id: str, expected_lang_code: str = 'en'):
    """
    Sends a prompt, evaluates, and displays agent's response.
    Uses the passed runner_instance.
    """
    if not all(f in globals() for f in ['is_movie_or_show_query', 'check_toxicity', 'check_pii', 'check_prompt_injection', 'check_rejection', 'check_has_ratings_and_emojis', 'check_has_sources', 'check_faithfulness', 'check_language']):
        print("🔴 ERROR: Not all evaluation helper functions are defined. Please run previous cells.")
        return None

    new_message = Content(role="user", parts=[Part(text=prompt_text)])
    print(f"🗣️ User: {prompt_text}")

    # --- Optional: Debugging check before run_async ---
    # print(f"  DEBUG (ask_movie_agent): Runner App: '{runner_instance.app_name}', Session Service ID: {id(runner_instance.session_service)}")
    # try:
    #     await runner_instance.session_service.get_session(app_name=runner_instance.app_name, user_id=current_user_id, session_id=current_session_id)
    #     print(f"    DEBUG (ask_movie_agent): Pre-run_async session check SUCCEEDED for session '{current_session_id}'.")
    # except Exception as e_get_debug:
    #     print(f"    DEBUG (ask_movie_agent): Pre-run_async session check FAILED for session '{current_session_id}'. Error: {e_get_debug}")
    # --- End Debugging Check ---

    print("⏳ Agent thinking...")
    raw_agent_response_text = ""
    event_count = 0
    tool_usage_info = []

    try:
        async for event in runner_instance.run_async(user_id=current_user_id, session_id=current_session_id, new_message=new_message):
            event_count += 1
            if hasattr(event, 'content') and event.content and hasattr(event.content, 'parts'):
                text_parts = [part.text for part in event.content.parts if hasattr(part, 'text') and part.text]
                raw_agent_response_text = "\n".join(text_parts)
                if hasattr(event.content.parts[0], 'function_call') and event.content.parts[0].function_call:
                     fc = event.content.parts[0].function_call
                     args_str = ", ".join([f"{k}={v}" for k, v in fc.args.items()]) if isinstance(fc.args, dict) else str(fc.args)
                     tool_usage_info.append(f"🛠️ Agent used tool: {fc.name}({args_str})")
                elif hasattr(event, 'grounding_metadata') and event.grounding_metadata:
                    if event.grounding_metadata.web_search_queries:
                        tool_usage_info.append(f"🛠️ Agent used Google Search with queries: {event.grounding_metadata.web_search_queries}")
            elif hasattr(event, 'error_message') and event.error_message:
                raw_agent_response_text = f"🚨 Agent Error: {event.error_message}"
                if hasattr(event, 'error_code'): raw_agent_response_text += f" (Code: {event.error_code})"
            elif isinstance(event, str) and event_count == 1: raw_agent_response_text = event # Unlikely with current ADK
            if raw_agent_response_text: break # Process first substantive event
    except ValueError as ve:
        if "Session not found" in str(ve):
            print(f"🔴 FATAL (ask_movie_agent): Session '{current_session_id}' not found by runner.run_async.")
            print(f"   Runner App: '{runner_instance.app_name}', User: '{current_user_id}', Session Service ID: {id(runner_instance.session_service)}")
            print(f"   Ensure session was created with these exact parameters on this session service instance.")
            raw_agent_response_text = f"ERROR: Session '{current_session_id}' could not be found by the agent runner."
            # To prevent further errors in eval, we set text but safety_breach will be an issue.
        else:
            print(f"🔴 ValueError in runner.run_async: {ve}")
            raw_agent_response_text = f"ERROR: A ValueError occurred: {ve}"
        print(traceback.format_exc())
    except Exception as e_run:
        print(f"🔴 Unexpected error in runner.run_async: {e_run}")
        raw_agent_response_text = f"ERROR: An unexpected error occurred: {e_run}"
        print(traceback.format_exc())


    # --- Response Evaluation Stage ---
    # (Identical to your last good version of this section)
    print("\n🔍 Evaluating agent response...")
    safety_breach = False
    feedback_messages = []

    if not raw_agent_response_text or "ERROR:" in raw_agent_response_text: # Handle if run_async failed
        final_display_text = raw_agent_response_text if raw_agent_response_text else "✨ Agent Response: (No textual content was generated or runner failed)"
        if raw_agent_response_text and "ERROR:" in raw_agent_response_text:
            feedback_messages.append(f"CRITICAL: ⚠️ {raw_agent_response_text}") # Log the error
        elif event_count == 0 and not raw_agent_response_text :
            feedback_messages.append("INFO: No events received from the agent.")
        else:
            feedback_messages.append(f"INFO: No textual content extracted from {event_count} event(s).")
    else:
        # 1. Safety Checks
        if not check_toxicity(raw_agent_response_text):
            feedback_messages.append("CRITICAL: ⚠️ Toxicity check failed.")
            safety_breach = True
        if not check_pii(raw_agent_response_text):
            feedback_messages.append("CRITICAL: ⚠️ PII check failed.")
            safety_breach = True
        if not check_prompt_injection(raw_agent_response_text):
            feedback_messages.append("CRITICAL: ⚠️ Prompt injection / security check failed.")
            safety_breach = True

        if safety_breach:
            final_display_text = "🛡️ For your safety, the agent's response has been withheld due to potential issues."
            # feedback_messages are already populated with critical failure reasons
        else:
            # 2. Content and Format Checks
            current_prompt_is_explicitly_movie_query = is_movie_or_show_query(prompt_text)
            agent_declined = "sorry, i only answer questions about movies and tv shows" in raw_agent_response_text.lower()
            agent_could_not_find = "couldn't find any reviews" in raw_agent_response_text.lower()

            if not agent_declined and check_has_ratings_and_emojis(raw_agent_response_text):
                is_topic_movie_for_rejection_check = True
            else:
                is_topic_movie_for_rejection_check = current_prompt_is_explicitly_movie_query

            if not check_rejection(raw_agent_response_text, is_topic_movie_for_rejection_check):
                feedback_messages.append(f"INFO: ⚠️ Rejection check issue. Prompt: '{prompt_text}'. Effective topic: {'movie-related' if is_topic_movie_for_rejection_check else 'not movie-related'}.")

            is_actual_movie_review_provided = not agent_declined and \
                                              not agent_could_not_find and \
                                              check_has_ratings_and_emojis(raw_agent_response_text)

            if is_actual_movie_review_provided:
                if not check_has_sources(raw_agent_response_text):
                    feedback_messages.append("INFO: ⚠️ Content: Missing review sources or links.")
                if not check_faithfulness(raw_agent_response_text):
                    feedback_messages.append("INFO: ⚠️ Content: Did not cite known review sources.")

            if not check_language(raw_agent_response_text, expected_lang_code=expected_lang_code):
                detected_lang = "unknown"; L = locals(); exec("try: detected_lang = detect(raw_agent_response_text)\nexcept: pass", globals(), L); detected_lang=L['detected_lang']
                feedback_messages.append(f"INFO: ⚠️ Language check failed: Expected '{expected_lang_code}', detected '{detected_lang}'.")

            if not feedback_messages: # If no issues found yet
                feedback_messages.append("✅ All non-critical checks passed!")
            final_display_text = f"✨ Agent Response:\n{raw_agent_response_text}"

    # --- Display ---
    if tool_usage_info:
        for info in tool_usage_info: print(info)
    print("\n--- Evaluation Summary ---")
    for msg in feedback_messages: print(msg)
    print("--------------------------")
    display(Markdown(final_display_text))
    print("\n" + "-" * 70)

    return raw_agent_response_text if not safety_breach and "ERROR:" not in raw_agent_response_text else None

print("`ask_movie_agent` helper function defined.")

`ask_movie_agent` helper function defined.


In [26]:
# @title Helper function to display session history

async def show_session_history(runner_instance: Runner, current_user_id: str, current_session_id: str):
    """
    Retrieves and prints the formatted conversation history for a given session.
    Uses the passed runner_instance to access its session_service.
    """
    # (Your refined show_session_history function code goes here)
    # Ensure it uses runner_instance.app_name and runner_instance.session_service
    print(f"\n--- Session History (App: {runner_instance.app_name}, User: {current_user_id}, Session: {current_session_id}) ---")
    try:
        session = await runner_instance.session_service.get_session(
            app_name=runner_instance.app_name, # Use app_name from the runner
            user_id=current_user_id,
            session_id=current_session_id
        )
        if hasattr(session, 'events') and session.events:
            for idx, event in enumerate(session.events):
                print(f"Event {idx + 1}:")
                author = getattr(event, 'author', getattr(event.content, 'role', "Unknown"))
                print(f"  Author/Role: {author}")
                if hasattr(event, 'content') and event.content and hasattr(event.content, 'parts'):
                    for p_idx, part in enumerate(event.content.parts):
                        if hasattr(part, 'text') and part.text: print(f"  Part {p_idx+1} (Text): {part.text.strip()}")
                        # Add function_call / function_response display if needed
                if hasattr(event, 'grounding_metadata') and event.grounding_metadata and event.grounding_metadata.web_search_queries:
                    print(f"    Grounding: Web Search: {event.grounding_metadata.web_search_queries}")
                print("  ---")
        elif hasattr(session, 'events') and not session.events :
             print("No events (messages) in this session yet.")
        else: # session_object has no 'events'
            print(f"Session object (id: {session.id}) does not have an 'events' attribute or it's not as expected.")
            print(f"DEBUG: Session attributes: {vars(session).keys() if hasattr(session, '__dict__') else 'N/A (not a standard object)'}")

    except Exception as e:
        print(f"🔴 Error retrieving session history for '{current_session_id}': {e}")
        print(traceback.format_exc())
    print("--------------------------------------------------------------------")

print("`show_session_history` helper function defined.")

`show_session_history` helper function defined.


In [27]:
# @title Example Stateful Conversation

# This cell runs a stateful conversation test.
# It uses the 'runner' and 'session_service' initialized and the 'APP_NAME_MAIN'

async def run_main_conversation_flow():
    """
    Main function to run the stateful conversation test sequence.
    """
    if not all(k in globals() for k in ['runner', 'session_service', 'APP_NAME_MAIN', 'ask_movie_agent', 'show_session_history']):
        print("🔴 ERROR: Essential components not defined. Please run all setup cells.")
        return

    USER_ID_TEST = "test_user_002" # Using a new user ID for a fresh start if needed
    SESSION_ID_TEST = "test_session_multi_001" # Fresh ID for this test run

    print(f"--- Test Setup ---")
    print(f"Using Runner App Name: '{runner.app_name}' (should be '{APP_NAME_MAIN}')")
    # ... (rest of the initial checks from your working version) ...
    if runner.app_name != APP_NAME_MAIN or runner.session_service is not session_service:
        print(f"🔴 CRITICAL Mismatch: Runner config: App='{runner.app_name}', ServiceID={id(runner.session_service)}. Expected App='{APP_NAME_MAIN}', ServiceID={id(session_service)}")
        return
    else:
        print(f"✅ Runner configuration verified (App: '{runner.app_name}', ServiceID: {id(runner.session_service)}).")


    print(f"\nEnsuring session: App='{runner.app_name}', User='{USER_ID_TEST}', Session='{SESSION_ID_TEST}'")
    try:
        await session_service.create_session(
            app_name=runner.app_name, user_id=USER_ID_TEST, session_id=SESSION_ID_TEST
        )
        s = await session_service.get_session(app_name=runner.app_name, user_id=USER_ID_TEST, session_id=SESSION_ID_TEST)
        print(f"✅ Session '{s.id}' is ready for testing.")
    except Exception as e:
        print(f"🔴 ERROR setting up test session '{SESSION_ID_TEST}': {e}")
        print(traceback.format_exc())
        return

    print(f"\n🎬 Starting Conversation (Session: {SESSION_ID_TEST})...\n")

    # --- English Test Queries (as before) ---
    english_queries = [
        ("Tell me about the movie 'Oppenheimer'.", 'en'),
        ("How were the visuals in that film?", 'en'),
        ("Okay, now what about reviews for 'Barbie'?", 'en'),
        ("And its overall rating?", 'en'),
        ("What's the capital of France?", 'en'),
        ("Can you elaborate on its story?", 'en'), # Should refer to Barbie
        ("Was it good?", 'en'), # Should refer to Barbie
    ]

    print("\n--- Running English Queries ---")
    for i, (q_text, q_lang) in enumerate(english_queries):
        print(f"\n--- Interaction E{i+1} ---")
        await ask_movie_agent(runner, q_text, USER_ID_TEST, SESSION_ID_TEST, expected_lang_code=q_lang)
        await show_session_history(runner, USER_ID_TEST, SESSION_ID_TEST)
        if i < len(english_queries) - 1: await asyncio.sleep(0.2)

    # --- Multilingual Test Queries ---
    multilingual_queries = [
        ("¿Qué tal la película 'Coco'?", 'es'), # Spanish query for an English language film title (usually)
        ("Parle-moi du film 'Amélie'.", 'fr'),   # French query for a French film
        ("레미제라블 영화에 대해 알려줘", 'ko'), # Korean query for "Les Misérables" (agent might search for Korean title or transliterated)
        ("Wie war der Film 'Das Leben der Anderen'?", 'de') # German query for a German film
    ]

    print("\n\n--- Running Multilingual Queries ---")
    for i, (q_text, q_lang) in enumerate(multilingual_queries):
        print(f"\n--- Interaction M{i+1} (Lang: {q_lang}) ---")
        # The agent will attempt to respond in the query language for conversational parts.
        # The review content itself might be primarily English-based depending on search results.
        # The `check_language` in `ask_movie_agent` will evaluate the agent's main response language.
        await ask_movie_agent(runner, q_text, USER_ID_TEST, SESSION_ID_TEST, expected_lang_code=q_lang)
        await show_session_history(runner, USER_ID_TEST, SESSION_ID_TEST)
        if i < len(multilingual_queries) - 1: await asyncio.sleep(0.2)

    print("\n🏁 Conversation test sequence complete.")

# --- Execute the Test Flow ---
await run_main_conversation_flow()

✅ Session service verified: Successfully created and retrieved test session 'service_check_session'.
--- Test Setup ---
Using Runner App Name: 'critics_cut_app_v1' (should be 'critics_cut_app_v1')
✅ Runner configuration verified (App: 'critics_cut_app_v1', ServiceID: 133446741465744).

Ensuring session: App='critics_cut_app_v1', User='test_user_002', Session='test_session_multi_001'
✅ Session 'test_session_multi_001' is ready for testing.

🎬 Starting Conversation (Session: test_session_multi_001)...


--- Running English Queries ---

--- Interaction E1 ---
🗣️ User: Tell me about the movie 'Oppenheimer'.
⏳ Agent thinking...

🔍 Evaluating agent response...


🛠️ Agent used Google Search with queries: ['Oppenheimer movie review']

--- Evaluation Summary ---
✅ All non-critical checks passed!
--------------------------


✨ Agent Response:
**Oppenheimer – Review Summary**

Christopher Nolan's "Oppenheimer" tells the story of J. Robert Oppenheimer, the scientist who developed the atomic bomb, and his later struggles with the consequences of his creation. Reviews highlight the film's mature storytelling, impressive visuals, and strong performances, particularly by Cillian Murphy as Oppenheimer and Robert Downey Jr. as Lewis Strauss. The film utilizes a non-linear narrative, shifting between color and black-and-white sequences to represent different perspectives and timelines.

*   **Acting:** ⭐⭐⭐⭐⭐⭐⭐⭐⭐☆ (9/10) 🎭 Cillian Murphy's portrayal of Oppenheimer is widely praised for capturing the character's complexities and tortured psyche. Robert Downey Jr. also delivers a standout performance.
*   **Visuals/Cinematography:** ⭐⭐⭐⭐⭐⭐⭐⭐⭐☆ (9/10) 👁️ The film is visually striking, with Nolan using IMAX cameras and unique black-and-white film to create immersive and impactful images.
*   **Story/Plot:** ⭐⭐⭐⭐⭐⭐⭐⭐☆☆ (8/10) 📖 The narrative is dense and complex, exploring Oppenheimer's life, the development of the atomic bomb, and the moral implications of its use. The non-linear structure adds depth but may require close attention.
*   **Direction:** ⭐⭐⭐⭐⭐⭐⭐⭐⭐☆ (9/10) 🎬 Christopher Nolan's direction is masterful, combining large-scale storytelling with intimate character moments. The film is considered one of his most mature and ambitious works.
*   **Overall:** ⭐⭐⭐⭐⭐⭐⭐⭐⭐☆ (9/10) 🏆 "Oppenheimer" is a thought-provoking and visually stunning film that explores the life of a brilliant but troubled man and the weighty consequences of his creation. It's a film that demands to be reckoned with and leaves a lasting impact.

**Sources:**

*   **BBC:** Oppenheimer review: A 'magnificent' story of a tragic American genius](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFvHTIkp8de1bXCd5gnNLALSoxJtC8nD8eZOVcD8sUiBAuByHmNv9rRUxZcmiFuCqR5W9IPjOCl63saS-L6PHabBDtxFYHGPHq_8MHwbn5aSDc8Zi1ElhUTjF-Lz4_JUOPn-TGGid3AW03uivJkBQIV7kdHm4w66wiE4IfIhjNLkKnCsJ1p5ZIW9Dn3akbYCtX58g6tJS3m)
*   **Empire Online:** Oppenheimer – Review](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGveyxKr4Rj27FW8nAdGNFRlyv-PhCqt9oBHMtcaCksoIssIWVWnUPrVs6o5XRmjCXu5iiD6L7qoEuZVLMzp-jrYLp32pa1WW_KFbVw5rB4EHRgU0wLIq3RQlgLD0HFRCvJVoq6KmmwxwQ7vTDgcxE4qDAw)



----------------------------------------------------------------------

--- Session History (App: critics_cut_app_v1, User: test_user_002, Session: test_session_multi_001) ---
Event 1:
  Author/Role: user
  Part 1 (Text): Tell me about the movie 'Oppenheimer'.
  ---
Event 2:
  Author/Role: MovieReviewAgent
  Part 1 (Text): **Oppenheimer – Review Summary**

Christopher Nolan's "Oppenheimer" tells the story of J. Robert Oppenheimer, the scientist who developed the atomic bomb, and his later struggles with the consequences of his creation. Reviews highlight the film's mature storytelling, impressive visuals, and strong performances, particularly by Cillian Murphy as Oppenheimer and Robert Downey Jr. as Lewis Strauss. The film utilizes a non-linear narrative, shifting between color and black-and-white sequences to represent different perspectives and timelines.

*   **Acting:** ⭐⭐⭐⭐⭐⭐⭐⭐⭐☆ (9/10) 🎭 Cillian Murphy's portrayal of Oppenheimer is widely praised for capturing the character's co


🔍 Evaluating agent response...

--- Evaluation Summary ---
INFO: ⚠️ Content: Missing review sources or links.
INFO: ⚠️ Content: Did not cite known review sources.
--------------------------


...

✨ Agent Response:
알겠습니다. 레미제라블 영화에 대한 리뷰 요약입니다:

**레미제라블 – 리뷰 요약**

"레미제라블"은 빅토르 위고의 동명 소설을 원작으로 한 장편 영화로, 19세기 프랑스를 배경으로 장발장의 삶과 투쟁을 그린다. 리뷰에서는 영화의 웅장한 스케일, 강력한 연기, 그리고 시대를 초월하는 주제를 칭찬합니다.

*   **연기:** ⭐⭐⭐⭐⭐⭐⭐⭐☆☆ (8/10) 🎭 휴 잭맨은 장발장 역을 훌륭하게 소화했으며, 앤 해서웨이는 팡틴 역으로 아카데미 여우조연상을 수상했습니다.
*   **영상/촬영:** ⭐⭐⭐⭐⭐⭐⭐⭐☆☆ (8/10) 👁️ 영화는 19세기 프랑스의 분위기를 생생하게 담아냈으며, 웅장한 스케일의 전투 장면과 아름다운 배경이 인상적입니다.
*   **스토리/줄거리:** ⭐⭐⭐⭐⭐⭐⭐⭐☆☆ (8/10) 📖 이야기는 장발장의 삶과 프랑스 사회의 부조리를 다루며, 감동과 여운을 남깁니다.
*   **연출:** ⭐⭐⭐⭐⭐⭐⭐⭐☆☆ (8/10) 🎬 톰 후퍼 감독은 원작 소설의 감동을 스크린에 잘 옮겼으며, 배우들의 연기를 훌륭하게 이끌어냈습니다.
*   **종합:** ⭐⭐⭐⭐⭐⭐⭐⭐☆☆ (8/10) 🏆 "레미제라블"은 감동적이고 웅장한 영화로, 시대를 초월하는 주제와 강력한 메시지를 전달합니다.

**출처:**

*   **Empire:** [https://www.empireonline.com/movies/reviews/les-miserables-review/](https://www.empireonline.com/movies/reviews/les-miserables-review/)
*   **The Guardian:** [https://www.theguardian.com/film/2013/jan/11/les-miserables-film-review](https://www.theguardian.com/film/2013/jan/11/les-miserables-film-review)



----------------------------------------------------------------------

--- Session History (App: critics_cut_app_v1, User: test_user_002, Session: test_session_multi_001) ---
Event 1:
  Author/Role: user
  Part 1 (Text): Tell me about the movie 'Oppenheimer'.
  ---
Event 2:
  Author/Role: MovieReviewAgent
  Part 1 (Text): **Oppenheimer – Review Summary**

Christopher Nolan's "Oppenheimer" tells the story of J. Robert Oppenheimer, the scientist who developed the atomic bomb, and his later struggles with the consequences of his creation. Reviews highlight the film's mature storytelling, impressive visuals, and strong performances, particularly by Cillian Murphy as Oppenheimer and Robert Downey Jr. as Lewis Strauss. The film utilizes a non-linear narrative, shifting between color and black-and-white sequences to represent different perspectives and timelines.

*   **Acting:** ⭐⭐⭐⭐⭐⭐⭐⭐⭐☆ (9/10) 🎭 Cillian Murphy's portrayal of Oppenheimer is widely praised for capturing the character's co


--- Evaluation Summary ---
✅ All non-critical checks passed!
--------------------------


✨ Agent Response:
Klar, hier ist eine Zusammenfassung der Kritiken für 'Das Leben der Anderen':

**Das Leben der Anderen – Review Summary**

'Das Leben der Anderen' ist ein deutscher Film, der im Ost-Berlin der 1980er Jahre spielt und die Geschichte eines Stasi-Hauptmanns erzählt, der eine Theaterschauspielerin und ihren Partner überwacht. Im Laufe der Zeit beginnt er, Empathie für sie zu entwickeln und ihr Leben zu schützen. Die Kritiken loben den Film für seine spannungsgeladene Handlung, die starken Leistungen und die bewegende Darstellung des Lebens in der DDR.

*   **Schauspiel:** ⭐⭐⭐⭐⭐⭐⭐⭐⭐☆ (10/10) 🎭 Ulrich Mühe liefert eine herausragende Leistung als Stasi-Hauptmann Gerd Wiesler ab.
*   **Visuell/Kamera:** ⭐⭐⭐⭐⭐⭐⭐⭐☆☆ (8/10) 👁️ Der Film fängt die Atmosphäre des düsteren und überwachten Ost-Berlins hervorragend ein.
*   **Geschichte/Handlung:** ⭐⭐⭐⭐⭐⭐⭐⭐⭐☆ (9/10) 📖 Die Geschichte ist fesselnd und regt zum Nachdenken über Überwachung, Moral und Menschlichkeit an.
*   **Regie:** ⭐⭐⭐⭐⭐⭐⭐⭐⭐☆ (9/10) 🎬 Florian Henckel von Donnersmarck führte Regie und schrieb das Drehbuch für diesen beeindruckenden Film.
*   **Gesamt:** ⭐⭐⭐⭐⭐⭐⭐⭐⭐☆ (9/10) 🏆 'Das Leben der Anderen' ist ein bewegender und wichtiger Film, der einen Einblick in das Leben in der DDR gibt und zum Nachdenken über die Bedeutung von Freiheit und Mitmenschlichkeit anregt.

**Quellen:**

*   **Roger Ebert:** [https://www.rogerebert.com/reviews/the-lives-of-others-2007](https://www.rogerebert.com/reviews/the-lives-of-others-2007)
*   **The Guardian:** [https://www.theguardian.com/film/2007/feb/09/drama.germany](https://www.theguardian.com/film/2007/feb/09/drama.germany)



----------------------------------------------------------------------

--- Session History (App: critics_cut_app_v1, User: test_user_002, Session: test_session_multi_001) ---
Event 1:
  Author/Role: user
  Part 1 (Text): Tell me about the movie 'Oppenheimer'.
  ---
Event 2:
  Author/Role: MovieReviewAgent
  Part 1 (Text): **Oppenheimer – Review Summary**

Christopher Nolan's "Oppenheimer" tells the story of J. Robert Oppenheimer, the scientist who developed the atomic bomb, and his later struggles with the consequences of his creation. Reviews highlight the film's mature storytelling, impressive visuals, and strong performances, particularly by Cillian Murphy as Oppenheimer and Robert Downey Jr. as Lewis Strauss. The film utilizes a non-linear narrative, shifting between color and black-and-white sequences to represent different perspectives and timelines.

*   **Acting:** ⭐⭐⭐⭐⭐⭐⭐⭐⭐☆ (9/10) 🎭 Cillian Murphy's portrayal of Oppenheimer is widely praised for capturing the character's co